## Create database connection engine

In [1]:
from sqlalchemy import create_engine
from configparser import ConfigParser


def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

params = config()

DB_URL = (
    f"postgresql+psycopg2://{params['user']}:{params['password']}"
    f"@{params['host']}:{params['port']}/{params['dbname']}"
)

# Create an engine and connect to the database
engine = create_engine(DB_URL, echo=False, future=True)

Create a session to interact with the database

In [2]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

# CREATING CLASSES AND CRUD OPERATIONS

In [3]:
from sqlalchemy.orm import relationship
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.orm import declarative_base

Base = declarative_base()

## ESG_CATEGORY CLASS

In [4]:
class ESGCategory(Base):
    __tablename__ = "esg_categories"

    id = Column(Integer, primary_key=True)
    label = Column(String(20), unique=True)
    category_name = Column(Text)
    description = Column(Text)

    def __repr__(self):
        return (
            f"<ESGCategory(id={self.id}, "
            f"label={self.label!r}, "
            f"category_name={self.category_name!r}, "
            f"description={self.description!r})>"
        )

CRUD OPERATIONS

In [5]:
# Create
new_cat = ESGCategory(label="T", category_name="Test", description="Test topics")
session.add(new_cat)
session.commit()

# READ OPERATION
cat = session.query(ESGCategory).filter_by(label="T").first()
print(f"Succesfully created and read {cat}","\n")

# UPDATE OPERATION
if cat:
    try:
        cat.category_name = "I Changed!"
        session.commit()
        print(f"Successfully updated = {cat}","\n")
    except:
        print("something went wrong")

# DELETE OPERATION
if cat:
    try:
        session.delete(cat)
        session.commit()
        print(f"Successfully deleted id={cat.id}") 
    except:
        print("something went wrong")
     

Succesfully created and read <ESGCategory(id=15, label='T', category_name='Test', description='Test topics')> 

Successfully updated = <ESGCategory(id=15, label='T', category_name='I Changed!', description='Test topics')> 

Successfully deleted id=15


## PLATFORM CLASS

In [6]:
class Platform(Base):
    __tablename__ = "platforms"

    id = Column(Integer, primary_key=True)
    name = Column(String(50), unique=True)

    def __repr__(self):
        return f"<Platform(id={self.id}, name={self.name!r})>"


In [7]:
# CREATE
new_platform = Platform(name="Twitter")
session.add(new_platform)
session.commit()

# READ
plat = session.query(Platform).filter_by(name="Twitter").first()
print("Read:", plat)

# UPDATE
if plat:
    plat.name = "X"
    session.commit()
    print("Updated:", plat)

# DELETE
if plat:
    session.delete(plat)
    session.commit()
    print(f"Deleted platform id={plat.id}")

Read: <Platform(id=7, name='Twitter')>
Updated: <Platform(id=7, name='X')>
Deleted platform id=7


## SENTIMENT_LABELS CLASS

In [8]:
class SentimentLabel(Base):
    __tablename__ = "sentiment_labels"

    id = Column(Integer, primary_key=True)
    label = Column(String(20), unique=True)
    description = Column(Text)

    def __repr__(self):
        return f"<SentimentLabel(id={self.id}, label={self.label!r}, description={self.description!r})>"

In [9]:
# CREATE
new_sentiment = SentimentLabel(label="positive", description="Positive sentiment")
session.add(new_sentiment)
session.commit()
print(f"Created: {new_sentiment}\n")

# READ
sent = session.query(SentimentLabel).filter_by(label="positive").first()
print(f"Read: {sent}\n")

# UPDATE
if sent:
    try:
        sent.description = "Optimistic / approving tone"
        session.commit()
        print(f"Updated: {sent}\n")
    except Exception as e:
        print("Update failed:", e)

# DELETE
if sent:
    try:
        session.delete(sent)
        session.commit()
        print(f"Deleted SentimentLabel id={sent.id}\n")
    except Exception as e:
        print("Delete failed:", e)

Created: <SentimentLabel(id=7, label='positive', description='Positive sentiment')>

Read: <SentimentLabel(id=7, label='positive', description='Positive sentiment')>

Updated: <SentimentLabel(id=7, label='positive', description='Optimistic / approving tone')>

Deleted SentimentLabel id=7



## ESG_Keywords Class

In [10]:
from sqlalchemy import Boolean

class ESGKeyword(Base):
    __tablename__ = "esg_keywords"

    id = Column(Integer, primary_key=True)
    keyword = Column(Text, nullable=False)
    category = Column(String(1))  # 'E' | 'S' | 'G'
    language = Column(String(10), default="el")
    active = Column(Boolean, default=True)  # Boolean stored as int
    inserted_at = Column(Text)           # could map to DateTime if preferred

    def __repr__(self):
        return (
            f"<ESGKeyword(id={self.id}, keyword={self.keyword!r}, "
            f"category={self.category!r}, language={self.language!r}, active={self.active})>"
        )


In [11]:
# CREATE
new_kw = ESGKeyword(keyword="climate change", category="E", language="en", active=True)
session.add(new_kw)
session.commit()
print(f"Created: {new_kw}\n")

# READ
kw = session.query(ESGKeyword).filter_by(keyword="climate change").first()
print(f"Read: {kw}\n")

# UPDATE
if kw:
    try:
        kw.keyword = "global warming"
        kw.active = False
        session.commit()
        print(f"Updated: {kw}\n")
    except Exception as e:
        print("Update failed:", e)

# DELETE
if kw:
    try:
        session.delete(kw)
        session.commit()
        print(f"Deleted ESGKeyword id={kw.id}\n")
    except Exception as e:
        print("Delete failed:", e)

Created: <ESGKeyword(id=196, keyword='climate change', category='E', language='en', active=True)>

Read: <ESGKeyword(id=196, keyword='climate change', category='E', language='en', active=True)>

Updated: <ESGKeyword(id=196, keyword='global warming', category='E', language='en', active=False)>

Deleted ESGKeyword id=196



In [12]:
session.query(ESGKeyword).count()

193

In [13]:
all_platforms = session.query(Platform).all()
all_platforms

[<Platform(id=1, name='Palowise')>,
 <Platform(id=2, name='Reddit')>,
 <Platform(id=3, name='Youtube')>]

## Social_Posts Class

In [14]:
from sqlalchemy import ForeignKey, DateTime

class SocialPost(Base):
    __tablename__ = "social_posts"

    post_id = Column(Integer, primary_key=True)
    post_text = Column(Text)
    esg_id = Column(Integer, ForeignKey("esg_categories.id"))
    sentiment_id = Column(Integer, ForeignKey("sentiment_labels.id"))
    platform_id = Column(Integer, ForeignKey("platforms.id"))
    engagement = Column(Integer)
    likes = Column(Integer)
    dislikes = Column(Integer)
    date_published = Column(DateTime)
    author = Column(String(150))
    url = Column(Text)
    description = Column(Text)
    other = Column(Text)
    date_insert = Column(DateTime)

    def __repr__(self):
        return (
            f"<SocialPost(post_id={self.post_id}, author={self.author!r}, "
            f"engagement={self.engagement}, platform_id={self.platform_id}, "
            f"esg_id={self.esg_id}, sentiment_id={self.sentiment_id})>"
        )

In [15]:
from datetime import datetime

# CREATE
new_post = SocialPost(
    post_text="This is a test ESG post",
    esg_id=1,               # must exist in esg_categories
    sentiment_id=1,         # must exist in sentiment_labels
    platform_id=1,          # must exist in platforms
    engagement=100,
    likes=80,
    dislikes=5,
    date_published=datetime(2025, 9, 2, 12, 0),
    author="John Doe",
    url="http://example.com/post/1",
    description="Demo description",
    other="Extra field"
)
session.add(new_post)
session.commit()
print("Created:", new_post, "\n")

# READ
post = session.query(SocialPost).filter_by(author="John Doe").first()
print("Read:", post, "\n")

# UPDATE
if post:
    try:
        post.engagement = 150
        session.commit()
        print("Updated:", post, "\n")
    except Exception as e:
        print("Update failed:", e)

# DELETE
if post:
    try:
        session.delete(post)
        session.commit()
        print(f"Deleted SocialPost id={post.post_id}\n")
    except Exception as e:
        print("Delete failed:", e)


Created: <SocialPost(post_id=89649, author='John Doe', engagement=100, platform_id=1, esg_id=1, sentiment_id=1)> 

Read: <SocialPost(post_id=89649, author='John Doe', engagement=100, platform_id=1, esg_id=1, sentiment_id=1)> 

Updated: <SocialPost(post_id=89649, author='John Doe', engagement=150, platform_id=1, esg_id=1, sentiment_id=1)> 

Deleted SocialPost id=89649



In [16]:
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker, Session

# make sure this uses the engine you already created
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

app = FastAPI(title="ESG CRUD API")
# ----------- ESGCategory Endpoints -----------

@app.post("/categories/")
def create_category(label: str, name: str, description: str, db: Session = Depends(get_db)):
    new_cat = ESGCategory(label=label, category_name=name, description=description)
    db.add(new_cat)
    db.commit()
    db.refresh(new_cat)
    return new_cat

@app.get("/categories/")
def get_categories(db: Session = Depends(get_db)):
    return db.query(ESGCategory).all()

@app.get("/categories/{cat_id}")
def get_category(cat_id: int, db: Session = Depends(get_db)):
    cat = db.query(ESGCategory).filter(ESGCategory.id == cat_id).first()
    if not cat:
        raise HTTPException(status_code=404, detail="Category not found")
    return cat

@app.put("/categories/{cat_id}")
def update_category(cat_id: int, name: str, description: str, db: Session = Depends(get_db)):
    cat = db.query(ESGCategory).filter(ESGCategory.id == cat_id).first()
    if not cat:
        raise HTTPException(status_code=404, detail="Category not found")
    cat.category_name = name
    cat.description = description
    db.commit()
    db.refresh(cat)
    return cat

@app.delete("/categories/{cat_id}")
def delete_category(cat_id: int, db: Session = Depends(get_db)):
    cat = db.query(ESGCategory).filter(ESGCategory.id == cat_id).first()
    if not cat:
        raise HTTPException(status_code=404, detail="Category not found")
    db.delete(cat)
    db.commit()
    return {"message": f"Category {cat_id} deleted"}

In [17]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()  # allow nested event loops in Jupyter

uvicorn.run(app, host="0.0.0.0", port=8000, reload=False)

INFO:     Started server process [24908]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [24908]
